## RNN

In [2]:
import numpy as np
import optuna
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from optuna import Trial, Study
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from utils.training import train, best_torch_device
from torchinfo import summary

%load_ext autoreload
%autoreload 2

In [3]:
device = best_torch_device()
# device = torch.device('cpu')
print(best_torch_device())

mps


In [4]:
X_train_valid = np.load("data/X_train_valid.npy")
y_train_valid = np.load("data/y_train_valid.npy")
person_train_valid = np.load("data/person_train_valid.npy")

X_train_aug = np.load("data/generated/frequency/X_train_augmented.npy")
y_train_aug = np.load("data/generated/frequency/y_train_augmented.npy")

X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")
person_test = np.load("data/person_test.npy")

y_train_valid -= 769
y_train_aug -= 769
y_test -= 769

X_train = X_train_valid[:1777]
y_train = y_train_valid[:1777]
X_valid = X_train_valid[1777:]
y_valid = y_train_valid[1777:]


In [5]:
X_train_aug.shape

(8885, 22, 1000)

In [6]:
class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

class Permute(nn.Module):
    def __init__(self, *dims):
        super(Permute, self).__init__()
        self.dims = dims

    def forward(self, x):
        return x.permute(*self.dims)
    
def train(model, optimizer, loader, val_loader, cel_loss, num_epochs):
    train_acc_hist = []
    val_acc_hist = []
    for epoch_idx in tqdm(range(num_epochs)):
        # Set model to train mode - useful for layers such as BatchNorm or Dropout whose behaviors change between train/eval
        model.train()
        train_count = 0
        train_correct_count = 0
        for batch_idx, (train_x, train_y) in enumerate(loader):
            train_x = train_x.float().to(device)
            train_y = train_y.long().to(device)
            optimizer.zero_grad()
            logits = model(train_x)
            loss = cel_loss(logits, train_y)
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                y_hat = torch.argmax(logits, dim=-1)
                train_correct_count += torch.sum(y_hat == train_y, axis=-1)
                train_count += train_x.size(0)

        train_acc = train_correct_count / train_count
        train_acc_hist.append(train_acc)

        model.eval()
        val_count = 0
        val_correct_count = 0
        with torch.no_grad():
            for idx, (val_x, val_y) in enumerate(val_loader):
                val_x = val_x.float().to(device)
                val_y = val_y.long().to(device)
                logits = model(val_x).detach()
                y_hat = torch.argmax(logits, dim=-1)
                val_correct_count += torch.sum(y_hat == val_y, axis=-1)
                val_count += val_x.size(0)
        val_acc = val_correct_count / val_count
        val_acc_hist.append(val_acc)
        print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

In [7]:
class RNN(nn.Module):

    def __init__(self):
        super(RNN, self).__init__()

        conv1_filter_num = 32
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv1_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv1_filter_num),
            nn.Dropout(0.3)
        )

        conv2_filter_num = 128
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=conv1_filter_num, out_channels=conv2_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv2_filter_num),
            nn.Dropout(0.3)
        )

        conv3_filter_num = 128
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=conv2_filter_num, out_channels=conv3_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv3_filter_num),
            nn.Dropout(0.3)
        )

        conv_out_filter_size = 19
        self.flatten = nn.Flatten(start_dim=2)
        self.linear1 = nn.Linear(in_features=conv_out_filter_size, out_features=conv_out_filter_size)

        self.flatten_cnn = nn.Flatten(start_dim=1)
        self.linear_cnn = nn.Linear(in_features=conv_out_filter_size*conv3_filter_num, out_features=4)

        self.tdd = nn.Linear(in_features=1000, out_features=50)

        conv_rnn_filter_num = 50
        self.conv_rnn = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv_rnn_filter_num, kernel_size=(64, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(7, 1)),
            nn.BatchNorm2d(conv_rnn_filter_num),
            nn.Dropout(0.3)
        )

        self.num_layers = 3
        self.hidden_size = 30
        self.lstm = nn.LSTM(input_size=conv3_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        # self.lstm = nn.LSTM(input_size=conv_rnn_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        
        self.linear2 = nn.Linear(in_features=self.hidden_size, out_features=4) # (N, 4)

        self.softmax = nn.Softmax(dim=1) # the dim corresponds to num_output_classes=4

    def forward(self, x):
        """
        x should have shape (N, H, L) = (N, 22, 1000), where
            L = sequence length
            N = batch size
            H = input size
        """
        (N, H, L) = x.shape

        x = x.view((-1, 22, 1000, 1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.flatten_cnn(x)
        # x = self.linear_cnn(x)

        # x = self.tdd(x)
        # x = x.view((-1, 22, 1000, 1))
        # x = self.conv_rnn(x)
        # x = self.flatten(x)
        x = x.permute(2, 0, 1) # permute to (L=time, N, H=channel_in)
        # print(x.shape)

        # device = torch.device('mps')
        # h0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # x, _ = self.lstm(x, (h0, c0))
        x, _ = self.lstm(x)
        x = self.linear2(x[-1]) # take the last time step
        
        x = self.softmax(x)
        return x

In [8]:
class RNN(nn.Module):

    def __init__(self):
        super(RNN, self).__init__()

        self.tdd = nn.Sequential(
            nn.Linear(22, 40),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

        self.bn = nn.BatchNorm1d(40)

        self.num_layers = 5 # 3, 4
        self.hidden_size = 20 # 20
        self.lstm = nn.LSTM(
            input_size=40,
            hidden_size=self.hidden_size,
            dropout=0.5,
            num_layers=self.num_layers,
            batch_first=True,
        )

        self.linear2 = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(20),
            nn.ReLU(),
            nn.BatchNorm1d(20),
            nn.Dropout(0.5),
            nn.LazyLinear(out_features=4),
        )

        self.softmax = nn.Softmax(dim=1)  # the dim corresponds to num_output_classes=4

    def forward(self, x):
        """
        x should have shape (N, H, L) = (N, 22, 1000), where
            L = sequence length
            N = batch size
            H = input size
        """
        x = x.permute(0, 2, 1) # (N, 1000, 22)
        x = self.tdd(x) # (N, 1000, 40)

        x = x.permute(0, 2, 1)
        x = self.bn(x)
        x = x.permute(0, 2, 1)

        # device = torch.device('mps')
        # h0 = torch.randn(self.num_layers, self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers, self.hidden_size).to(device)
        x, _ = self.lstm(x)

        x = self.linear2(x)

        x = self.softmax(x)
        return x

In [9]:
bsz = 128
sample_device = torch.device('mps')
sample_model = RNN().to(sample_device)
test_input = torch.randn(128, 22, 1000).to(sample_device)
print(sample_model(test_input).shape)
summary(sample_model, (bsz, 22, 1000), device=device)



/Users/yubo/miniforge3/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([128, 4])


Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [128, 4]                  --
├─Sequential: 1-1                        [128, 1000, 40]           --
│    └─Linear: 2-1                       [128, 1000, 40]           920
│    └─ReLU: 2-2                         [128, 1000, 40]           --
│    └─Dropout: 2-3                      [128, 1000, 40]           --
├─BatchNorm1d: 1-2                       [128, 40, 1000]           80
├─LSTM: 1-3                              [128, 1000, 20]           18,400
├─Sequential: 1-4                        [128, 4]                  --
│    └─Flatten: 2-4                      [128, 20000]              --
│    └─Linear: 2-5                       [128, 20]                 400,020
│    └─ReLU: 2-6                         [128, 20]                 --
│    └─BatchNorm1d: 2-7                  [128, 20]                 40
│    └─Dropout: 2-8                      [128, 20]                 --
│    

In [25]:
bsz = 128

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)), shuffle=True, batch_size=bsz)
train_aug_loader = DataLoader(TensorDataset(torch.from_numpy(X_train_aug), torch.from_numpy(y_train_aug)), shuffle=True, batch_size=bsz)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid)), shuffle=False)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)), shuffle=False)

print(len(train_loader), len(train_aug_loader), len(val_loader), len(test_loader))

print(X_train_aug.shape)

14 70 338 443
(8885, 22, 1000)


In [26]:
model = RNN().to(device)
print(model)
num_chans, sequence_length = 22, 1000  # Image dimensions
test_input = torch.randn(bsz, num_chans, sequence_length).to(device)
print(model(test_input).shape)
# pred = model(test_input)[0]
# print(nn.CrossEntropyLoss()(pred, torch.from_numpy(y_train[0:1])))

RNN(
  (tdd): Sequential(
    (0): Linear(in_features=22, out_features=40, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (bn): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(40, 20, num_layers=5, batch_first=True, dropout=0.5)
  (linear2): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=20, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): LazyLinear(in_features=0, out_features=4, bias=True)
  )
  (softmax): Softmax(dim=1)
)
torch.Size([128, 4])


In [27]:
cel_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.000005)
num_epochs = 100
loader = train_aug_loader

best_model = model
best_val = 0
train_acc_hist = []
val_acc_hist = []
for epoch_idx in tqdm(range(num_epochs)):
    # Set model to train mode - useful for layers such as BatchNorm or Dropout whose behaviors change between train/eval
    model.train()
    train_count = 0
    train_correct_count = 0
    for batch_idx, (train_x, train_y) in enumerate(loader):
        train_x = train_x.float().to(device)
        train_y = train_y.long().to(device)
        optimizer.zero_grad()
        logits = model(train_x)
        loss = cel_loss(logits, train_y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_hat = torch.argmax(logits, dim=-1)
            train_correct_count += torch.sum(y_hat == train_y, axis=-1)
            train_count += train_x.size(0)

    train_acc = train_correct_count / train_count
    train_acc_hist.append(train_acc)

    model.eval()
    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float().to(device)
            val_y = val_y.long().to(device)
            logits = model(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count
    val_acc_hist.append(val_acc)
    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

    if val_acc > best_val:
        best_val = val_acc
        best_model = model

  0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.254, Val acc: 0.243
Train acc: 0.279, Val acc: 0.263
Train acc: 0.337, Val acc: 0.320
Train acc: 0.359, Val acc: 0.334
Train acc: 0.371, Val acc: 0.393
Train acc: 0.395, Val acc: 0.376
Train acc: 0.419, Val acc: 0.373
Train acc: 0.427, Val acc: 0.388
Train acc: 0.451, Val acc: 0.405
Train acc: 0.461, Val acc: 0.393
Train acc: 0.471, Val acc: 0.405
Train acc: 0.481, Val acc: 0.402
Train acc: 0.493, Val acc: 0.373
Train acc: 0.501, Val acc: 0.402
Train acc: 0.500, Val acc: 0.379
Train acc: 0.513, Val acc: 0.414
Train acc: 0.532, Val acc: 0.388
Train acc: 0.524, Val acc: 0.361
Train acc: 0.524, Val acc: 0.373
Train acc: 0.552, Val acc: 0.376
Train acc: 0.559, Val acc: 0.393
Train acc: 0.558, Val acc: 0.361
Train acc: 0.558, Val acc: 0.379
Train acc: 0.558, Val acc: 0.373
Train acc: 0.571, Val acc: 0.349
Train acc: 0.588, Val acc: 0.396
Train acc: 0.582, Val acc: 0.408
Train acc: 0.591, Val acc: 0.405
Train acc: 0.593, Val acc: 0.370
Train acc: 0.609, Val acc: 0.355
Train acc:

KeyboardInterrupt: 

In [30]:
best_model.eval()
val_count = 0
val_correct_count = 0
with torch.no_grad():
    for idx, (val_x, val_y) in enumerate(test_loader):
        val_x = val_x.float().to(device)
        val_y = val_y.long().to(device)
        logits = best_model(val_x).detach()
        y_hat = torch.argmax(logits, dim=-1)
        val_correct_count += torch.sum(y_hat == val_y, axis=-1)
        val_count += val_x.size(0)
val_acc = val_correct_count / val_count

print("Test Accuracy:", val_acc.item())

Test Accuracy: 0.4063205420970917


In [29]:
print(best_val)

tensor(0.4142, device='mps:0')
